<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
! pip -q install transformers
! pip -q install parsivar

     |████████████████████████████████| 1.1MB 6.6MB/s 
     |████████████████████████████████| 1.1MB 22.5MB/s 
     |████████████████████████████████| 890kB 39.8MB/s 
     |████████████████████████████████| 3.0MB 43.9MB/s 
     |████████████████████████████████| 36.2MB 82kB/s 
     |████████████████████████████████| 1.5MB 37.0MB/s 


In [ ]:
import torch
from __future__ import unicode_literals
# from hazm import *
from parsivar import *
from pprint import pprint
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import spatial
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForTokenClassification
import re
parsivar_tokenizer = Tokenizer()
parsivar_normalizer = Normalizer(statistical_space_correction=False)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model = AutoModelForTokenClassification.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model.eval()
print()

# **Tokenize**

### Example

In [ ]:
tweet_example = """ بابا بیخیال :)) همون سال اتلتیکو با گل افساید به بایرن رفت فینال.

مشکل داوری نداشتیم اون بازی که میگی.
فصل قبلش هم ۴ ۰ با هاینکس زدیم شما رو
"""

tweet_example2 = """
 محسن رضایی هم یک جوان ۲۶،۲۷ ساله بوده
مدیریت جنگ با مدیریت فعلی کشور که اکثرا افراد لیبرابی هستن که در جنگ هیچ نقشی نداشتن و یا نهایتا پشت جبهه مشغول بودن قابل مقایسه نیست

"""

tweet_example3 = """
سلام
خواهر من فکر کنم دو سالی سرفه امانش را بریده بود.خیلی کارها کرد برای درمانش اما درست نشد که نشد. کرونا هم که شروع شد، خیلی نگرانش بودیم. 
چند روز پیش با چند تا نیش زنبور عسل به اصطلاح زنبور درمانی کرد و با کمال تعجب دیگه سرفه نمی کنه.

"""

tweet_example4 = """
«اصول مشخص دموکراتیک» همین بس که مغز خودتون و رفقاتون رو گنده میپندارید. من باشم با اتفاقات اخیر امریکا که به لطف انتیفا تا همین امشب ادامه داره، توحش و حمله رو به کسان دیگر نسبت نمیدم. کامنتتون سراسر بی پرنسیبی بود توام با توهم مروج دموکراسی بودن.
"""

tweet_example5 = """
الان من باید میلان ‌می بودم
"""

tweet_example6 = """
 از ضربه کاستاس مانولاس ب بارسا بدتر بود
"""

tweet_example7 = """
ضربه‌کاستاس‌مانولاس‌ب
"""

In [ ]:
input_sentence = tokenizer.tokenize(tweet_example6)
for token in input_sentence:
  print(token)

▁از
▁ضربه
▁کاست
اس
▁م
انولاس
▁ب
▁بارسا
▁بدتر
▁بود


In [ ]:
for token in parsivar_tokenizer.tokenize_words(parsivar_normalizer.normalize(tweet_example7)):
  print(token)

ضربه‌کاستاس‌مانولاس‌ب


### Tokenize All tweets

In [ ]:
! wget "https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt"

--2020-10-19 09:23:19--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2698436 (2.6M) [text/plain]
Saving to: ‘extracted_data.txt.1’

extracted_data.txt. 100%[===================>]   2.57M  --.-KB/s    in 0.1s    

2020-10-19 09:23:19 (20.9 MB/s) - ‘extracted_data.txt.1’ saved [2698436/2698436]



In [ ]:
! cat extracted_data.txt | grep 'مانولاس'

  از ضربه کاستاس مانولاس ب بارسا بدتر بود
امیدوارم دوباره کاستاس مانولاس کار خودشو بکنه و اینبار تو نیوکمپ


In [ ]:
tweets_str = ""
with open("extracted_data.txt") as file:
  for line in file.readlines():
    tweets_str += line
tweets_str[:20]

'خرداد ۱۳۹۹ «کیهان» م'

In [ ]:
tweets = tweets_str.split("\n\n**************\n\n")
tweets[0], len(tweets)

('خرداد ۱۳۹۹ «کیهان» میراث سناتور مصطفی مصباح\u200cزاده ۷۸ ساله شد \nکیهان لندن تنها رسانه\u200c ایرانی در تبعید است که «از جنگ دوم جهانی تاکنون قدمت و یک مؤسسه\u200c غصب \u200cشده در تهران» دارد.\n\n',
 9349)

In [ ]:
len(tweets)

9349

In [ ]:
len(set(tweets))

9349

In [ ]:
tweets_tokens = []
for tweet in tweets:
  tweet_tokens = re.sub('[.]+[ ]?[.]+', '.', re.sub(' +', ' ',
                                    tweet.replace('_', ' ').
                                    replace('|', ' ').
                                    replace('/', ' ').
                                    replace('+', ' ').
                                    replace('[', ' ').
                                    replace(']', ' ')))
  tweet_tokens = parsivar_tokenizer.tokenize_words(parsivar_normalizer.normalize(tweet_tokens))
  tweets_tokens.append(tweet_tokens)

In [ ]:
tweets_tokens[0]

['خرداد',
 '1399',
 '«',
 'کیهان',
 '»',
 'میراث',
 'سناتور',
 'مصطفی',
 'مصباح\u200cزاده',
 '78',
 'ساله',
 'شد',
 '.',
 'کیهان',
 'لندن',
 'تنها',
 'رسانه',
 'ایرانی',
 'در',
 'تبعید',
 'است',
 'که',
 '«',
 'از',
 'جنگ',
 'دوم',
 'جهانی',
 'تاکنون',
 'قدمت',
 'و',
 'یک',
 'موسسه',
 'غصب',
 'شده',
 'در',
 'تهران',
 '»',
 'دارد',
 '.']

In [ ]:
counter = 0
for i in range(1, 11):
  with open('persian-ner-twitter-data' + str(i) + '.txt', 'w') as file:
    if i != 10:
      bin = tweets_tokens[(i-1)*1000:i*1000]
    else:
      bin = tweets_tokens[(i-1)*1000:len(tweets_tokens)]
    for tweet_tokens in bin:
      for token in tweet_tokens:
        if token != '':
          counter += 1
          file.write(token + '\tO\n')
      file.write('\n')
print(counter)

276026


In [ ]:
! head -n 50 persian-ner-twitter-data1.txt

خرداد	O
1399	O
«	O
کیهان	O
»	O
میراث	O
سناتور	O
مصطفی	O
مصباح‌زاده	O
78	O
ساله	O
شد	O
.	O
کیهان	O
لندن	O
تنها	O
رسانه	O
ایرانی	O
در	O
تبعید	O
است	O
که	O
«	O
از	O
جنگ	O
دوم	O
جهانی	O
تاکنون	O
قدمت	O
و	O
یک	O
موسسه	O
غصب	O
شده	O
در	O
تهران	O
»	O
دارد	O
.	O

چهارمین	O
قهرمانی	O
فلومیننسه	O
در	O
برزیل	O
:	O

زندگی	O
سلام	O
.	O


In [ ]:
! head -n 50 persian-ner-twitter-data5.txt

دوران	O
دفاع	O
مقدس	O
اوج	O
افتخارات	O
ملت	O
ایران	O
است	O
.	O
مقام	O
معظم	O
رهبری	O
.	O
هفته	O
دفاع	O
مقدس	O
.	O
غبارروبی	O
مزار	O
شهدای	O
روستای	O
ارتش‌آباد	O
.	O
زادگاه	O
شهید	O
طیب	O
حاج	O
رضایی	O
.	O
پنجشنبه.99	O
7	O
3	O

توییت‌لری	O
.	O
هی	O
برار‌جو	O
تا	O
بیه‌هی	O
روزگارو‌هی‌چنینه	O
.	O
جا	O
پاییز	O
و	O
جا	O
زمسّو	O
همیشه	O
بهار	O
نشینه	O
.	O


In [ ]:
! head -n 50 persian-ner-twitter-data10.txt

خرسند	O
.	O

هند	O
در	O
حال	O
مذاکره	O
با	O
اروپا	O
برای	O
ادامه	O
واردات	O
نفت	O
از	O
ایران	O
علی	O
رغم	O
تحریم‌ها	O
است	O
.«	O
هندوستان	O
تایمز	O
»	O
نوشت	O
:	O
.	O
طبق	O
گفته	O
منابع	O
رسمی	O
،	O
هند	O
جدا	O
از	O
تلاش	O
برای	O
مجاب	O
کردن	O
آمریکا	O
مبنی	O
بر‌اینکه	O
تحریم‌های	O
این	O
کشور	O
علیه	O
ایران	O
پس	O
از	O
خروج	O
از	O


# **Model**

In [ ]:
from transformers import pipeline

In [ ]:
pars_ner = pipeline('ner', model=model, tokenizer=tokenizer)
pars_ner(tweet_example)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-organization',
  'index': 8,
  'score': 0.9995940327644348,
  'word': 'اتلتیکو'},
 {'entity': 'B-organization',
  'index': 13,
  'score': 0.9972712397575378,
  'word': 'بایرن'},
 {'entity': 'B-person',
  'index': 31,
  'score': 0.9915841817855835,
  'word': 'هاینکس'}]